**Marcus Skinner** <br>
**CSM 4370: Data Science** <br>
**Professor Abdul** <br>

<h1>Problem Statement</h1>
I want to create an application that will help students and instructors organize and plan their courses at Eastern Illinois University. The application will take into account the student's completed courses, required courses, and the prerequisites for the courses.

<h1>1. Data Acquisition and Cleaning</h1>

We need the following information about all courses offered at EIU to implement the application:

- Course numbers
- Course names
- Course descriptions
- Course prerequisites

We can find all this information on the course catalog at https://catalog.eiu.edu/. We will need to scrape, parse, and clean the data from the catalog. To do this, we will use the following python packages:
- BeautifulSoup4
- urlopen
- re 
- String

<h2>1.1 Methods</h2>

The catalog has 25 pages of courses, which we'll refer to as **directory pages**. A directory in the catalog looks like:

![Output](imgs/catalog_page.png)

Notice, each course has a link. A link brings us to a **course page** that looks like: 

![Output](imgs/course_page.png)

In our main method, we will loop through every directory page and scrape it. All directory page URLs have the form:

https://catalog.eiu.edu/content.phpvcatoid=38&catoid=38&navoid=1742&filter%5Bitem_type%5D=3&filter%5Bonly_active%5D=
1&filter%5B3%5D=1&filter%5Bcpage%5D= + **X** + #acalog_template_course_filter

where **X** is the directory page number. We can increment X to iterate through each directory page URL and use the getCourses method to scrape each course page listed on a directory page. The code for our main method is given below.

In [ ]:
# import relevant modules
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re

def main():
    # Initialize CSV File
    filename = "courses.csv"
    f = open(filename, "a")
    headers = "Course_Number, Course_Name, Prerequisites \n"
    f.write(headers)

    for i in range(1, 26):
        my_url = "https://catalog.eiu.edu/content.php?catoid=38&catoid=38&navoid=1742&filter%5Bitem_type%5D=3&filter"\
                 "%5Bonly_active%5D=1&filter%5B3%5D=1&filter%5Bcpage%5D=" + str(i) + "#acalog_template_course_filter"

        getCourses(my_url, f)

    f.close()

We use the **getCourses** method to iterate through every link on a given directory page and write the course information to a csv file. We can find all the information about a course in a **td** tag with class **block_content**. In the heading, **h1**, we can find the course number and title. Here, we split on a "-" and write the number and title to a csv. For the prerequsites, we call the **cleanText** method to process the "Prerequsites & Notes" section of a course page.

In [ ]:
def getCourses(my_url, file):
    # Get webpage
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()

    # HTML parsing
    page_soup = soup(page_html, "html.parser")

    # Grabs each course link
    course_links = page_soup.findAll("td", {"class": "width"})

    # Go through each course link, open its page, and parse its data
    for course in course_links:
        # Open course page
        link = "https://catalog.eiu.edu/" + course.a.get("href")
        print(link)
        client = uReq(link)
        html = client.read()
        client.close()

        # Get data from webpage
        page_soup = soup(html, "html.parser")
        data = page_soup.find("td", {"class": "block_content"})

        # Get course number and title
        text = data.h1.text.split("-")
        course_number = text[0]
        course_title = text[1]

        # Get prerequisites
        text = data.get_text().replace("\n", "")
        course_prereqs = cleanText(text)

        # Write data to CSV file
        file.write(course_number + "," + course_title + "," + course_prereqs + "\n")

The processing for the prerequisites is a bit more complicated because there's a lot of noise around the information we need. To solve this, we use the **regular expressions** (re) python package to match patterns that look like course numbers. The regular expression for course number is r'[A-Z]{3,4} [1-4]\d{3}[G]?'

In [ ]:
def cleanText(string):
    pattern = re.compile(r'[A-Z]{3,4} [1-4]\d{3}[G]?')
    matches = pattern.finditer(string)
    prereqs = ""

    for match in matches:
        prereqs = prereqs + " " + match.group(0)

    return prereqs

Then finally, we can execute the entire program with:

In [ ]:
if __name__ == "__main__":
    main()

<h2>1.2 Known Issues</h2>

- For more accuracy on schedule planning, we will want to know when a course is offered. Some courses are offered in the Fall, Spring, every other year, on demand, or both in the Spring and Fall. We will need to account for all these cases.
- The prerequisites aren't as simple as a list. Some prerequisites take a form like "MAT 1402G or MAT 1400" or "MAT 1400 (can be taken concurrentely" or "Requires a 'C' or better in MAT 1400." We'll have to use a more specific set of regular expressions.
- Also, the first entry in the prerequisites is the course itself

<h1>2. Data Modeling</h1>

We will use **Neo4j** to build a model of the course data. With the model, we'll be able to make queries about a student's course schedule. The relationships between the nodes can be represented with the following graph:

![Output](imgs/courses_model.svg)

<h2>2.1 Sample Data</h2>

For simplicity, we will also use a cleaned, small sample of courses for demonstration. The courses in our sample will be the required courses for the Computer Science major at Eastern (excluding electives). The data is as follows

In [2]:
import pandas as pd

df = pd.read_csv("cs_courses.csv", encoding='cp1252')
print(df)

      Number                                         Title  Credits Semester  \
0   MAT1441G            Calculus and Analytical Geometry I        5      F S   
1    CSM2170                            Computer Science I        4      F S   
2    MAT2345              Elements of Discrete Mathematics        3        F   
3    MAT2442           Calculus and Analytical Geometry II        5      F S   
4    MAT2550                Introduction to Linear Algebra        3      F S   
5    CSM2670                   Object Oriented Programming        4        S   
6    CSM3570                           Numberical Analysis        3        S   
7    CSM3670                Principles of Computer Systems        3        F   
8    MAT3701                  Probability and Statistics I        3        F   
9    CSM3770                       Combinatorial Computing        3        S   
10   CSM3870                               Data Structures        3        F   
11   CSM3950             Introduction to

<h2>2.2 Neo4j Model</h2>

First, we have to connect to the courses database

In [ ]:
from neo4j import GraphDatabase

# These are not actual values. You will have to replace uri with your dataset location and the driver with the username and password
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("courses", "password"))

Then we will loop through every course in our dataframe and add it to the database. We will also add a computer science node and create a relationship between computer science and every course. 

In [ ]:
with driver.session() as session:
    # Create a node for the computer science department
    session.run('CREATE (d: Computer_Science {title: "Computer Science"})')
    # Loop through each course in df
    for i in range(len(df.index)):
        number = df.iloc[i,0]
        title = df.iloc[i,1]
        credits = df.iloc[i,2]
        semester = df.iloc[i,3]
        department = "Computer_Science"
        # Add a node for the course
        command = 'CREATE (c: %s {title: "%s", name: "%s", credits: "%s", semester: "%s"})' \
            % (number, number, title, credits, semester)
        session.run(command)
        # link the node to it's program(s)
        command = 'MATCH (c1: %s),(c2: %s) CREATE (c1)-[:REQUIRES]->(c2)' % (department, number)
        session.run(command)

Which results in the graph

![Output](imgs/graph1.png)

Next we will create the prerequsite relationships between courses.

In [ ]:
import re
# loop through each course in df
with driver.session() as session:
    for i in range(len(df.index)):
        number = df.iloc[i,0]
        # create a list of the prerequisites for the courses
        pattern = re.compile(r'[A-Z]{3,4}[1-4]\d{3}[G]?')
        string = df.iloc[i,4]
        prereqs = re.findall(pattern, string)
        # link the course to its prerequisites
        for req in prereqs:
            command = 'MATCH (c1: %s),(c2: %s) CREATE (c1)-[:NEEDS]->(c2)' \
                % (number, req)
            session.run(command)

Which gives the graph

![Output](imgs/graph2.png)

And create an example student named Ann--who has taken MAT1400, MAT1330, and MAT1441G

In [ ]:
with driver.session() as session:
    session.run('CREATE (p: Ann {title: "Ann"})')
    session.run('MATCH (p: Ann),(d:Computer_Science) CREATE (p)-[:BELONGS_TO]->(d)')
    courses_taken = ["MAT1400", "MAT1330", "MAT1441G"]
    
    for course in courses_taken:
        command = 'MATCH (c1: Ann),(c2: %s) CREATE (c1)-[:HAS_TAKEN]->(c2)' \
                % (course)
        session.run(command)

Which looks like

![Output](imgs/graph3.png)

We can use this model to demo some queries

<h1>3. Implementation</h1>
With our model, we can now query information about a studen't course schedule. We will use Ann as an example. For this demonstration, we will ask three main question:

- What courses can Ann take?
- What courses has Ann taken?
- What courses does Ann still need?

In [ ]:

with driver.session() as session:
    # What courses has Ann taken?
    question0 = session.run('MATCH p = (:Ann)-[:HAS_TAKEN]->(c) RETURN c.name as name')
    
    # What courses does Ann still need?
    question1 = session.run('MATCH p = (:Ann)-[:BELONGS_TO]->(d)-[:REQUIRES]->(courses_required) \
                                WHERE NOT (:Ann)-[:HAS_TAKEN]->(courses_required) \
                                RETURN courses_required.name as name')

    # What courses can Ann take?
    question2 = session.run('MATCH p = (:Ann)-[:BELONGS_TO]->(d)-[:REQUIRES]->(courses_required)-[:NEEDS]->(courses_needed)<-[:HAS_TAKEN]-(:Ann) \
                                WHERE NOT (:Ann)-[:HAS_TAKEN]->(courses_required) \
                                RETURN courses_required.name as name')
    
    # What courses can Ann take in the fall?
    question3 = session.run('MATCH p = (:Ann)-[:BELONGS_TO]->(d)-[:REQUIRES]->(courses_required)-[:NEEDS]->(courses_needed)<-[:HAS_TAKEN]-(:Ann) \
                                WHERE NOT (:Ann)-[:HAS_TAKEN]->(courses_required) \
                                AND (courses_required.semester = "F" OR courses_required.semester = "F S") \
                                RETURN courses_required.name as name')
    

In [ ]:
# What courses has Ann taken?
for record in question0:
    print(record['name'])

In [ ]:
# What courses does Ann still need?
for record in question1:
    print(record['name'])

In [ ]:
# What courses can Ann take?
for record in question2:
    print(record['name'])